# Submit Jobrun

## 1. Input Data

In [31]:
import requests
import json
from io import BytesIO
import pandas as pd
from zipfile import ZipFile

job_data = pd.read_csv("data/test_pseudopolicy_2.csv") #r"C:\_Working\PAT_20201019\debug\test_jobrun_input2.csv") #data/test_jobrun.csv')
job_data

,PseudoPolicyID,PolicyID,Limit,Participation,Retention,PolPrem,Stack,TIV,RatingGroup
0,902071_720820,902071,76030019.71,0.25,391213.71,1625918.698,720820,3.912137e+05,1
1,902071_720922,902071,76030019.71,0.25,386318.01,1625918.698,720922,3.863180e+05,1
2,902071_717188,902071,76030019.71,0.25,470508.19,1625918.698,717188,4.705082e+05,1
3,900324_701536,900324,76030019.71,0.25,537895.47,1625918.698,701536,5.378955e+05,1
4,902071_717311,902071,76030019.71,0.25,305977.33,1625918.698,717311,3.059773e+05,1
...,...,...,...,...,...,...,...,...,...
20991,900324_701182,900324,76030019.71,0.25,1318833.46,1625918.698,701182,1.318833e+06,11
20992,900324_701200,900324,76030019.71,0.25,729497.72,1625918.698,701200,7.294977e+05,11
20993,902071_716942,902071,76030019.71,0.25,2991696.49,1625918.698,716942,2.991696e+06,11
20994,902071_716970,902071,76030019.71,0.25,95359.53,1625918.698,716970,9.535953e+04,11


## 2. Input Parameters

In [29]:
job_para = json.load(open('data/test_api.json'))
print(json.dumps(job_para,indent=2))

{
  "data_source_type": "User_Upload",
  "valid_rules": 1073741826,
  "type_of_rating": "PSOLD",
  "curve_id": 2016,
  "psold": {
    "curve_persp": "Gross",
    "curve_coverage": "Building_Contents_BI",
    "peril_subline": "All_Perils",
    "trend_factor": 1.035,
    "average_accident_date": "1/1/2022",
    "blending_type": "missing_invalid",
    "blending_weights": [
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    "hpr_blending": false
  },
  "loss_alae_ratio": 1,
  "additional_coverage": 2,
  "deductible_treatment": "Retains_Limit",
  "coverage_type": "Building_Contents",
  "user_name": "Chengyou Xiao",
  "user_email": "chengyou.xiao@guycarp.com"
}


## 3. Request to the service

In [32]:
files = {
    'para': json.dumps(job_para),
    'data': job_data.to_csv(index=False)
}
resp = requests.post("http://frkcat-ws10:8002/api/job?jobrun=true", files=files)
if resp.status_code == 200:
    with ZipFile(BytesIO(resp.content),'r') as zf:
        df_res =  pd.read_csv(zf.open('pat_res.csv'))

df_res
# .sort_values('PseudoPolicyID')

# 20K records: 6s
# 188K: 28s
# 1m35s vs 26.3s

,PseudoPolicyID,PolicyID,Limit,Participation,Retention,PolPrem,Stack,TIV,RatingGroup,LossRatio,AOIr,PolLAS,DedLAS,Premium
0,902071_720820,902071,7.603002e+07,0.250000,391213.71,1.625919e+06,720820,3.912137e+05,1,1.0,24,16966.979683,15371.870103,126.993904
1,902071_720922,902071,7.603002e+07,0.250000,386318.01,1.625919e+06,720922,3.863180e+05,1,1.0,24,16952.016240,15348.934309,127.628619
2,902071_717188,902071,7.603002e+07,0.250000,470508.19,1.625919e+06,717188,4.705082e+05,1,1.0,24,17178.036989,15692.092038,118.302813
3,902071_717311,902071,7.603002e+07,0.250000,305977.33,1.625919e+06,717311,3.059773e+05,1,1.0,22,14923.759861,13875.084967,83.489762
4,902071_717384,902071,7.603002e+07,0.250000,486291.15,1.625919e+06,717384,4.862911e+05,1,1.0,24,17214.100360,15746.375703,116.852213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20991,884113_2071041,884113,1.309388e+08,0.315789,70293437.91,3.008146e+05,2071041,1.251127e+08,30,1.0,49,416767.160135,391620.213627,61149.192002
20992,884113_2071043,884113,1.309388e+08,0.315789,70293437.91,3.008146e+05,2071043,1.830085e+08,30,1.0,50,418209.352322,393062.405814,61149.192003
20993,884113_2071040,884113,1.309388e+08,0.315789,70293437.91,3.008146e+05,2071040,1.882963e+08,30,1.0,51,420808.901084,395661.438220,61150.447612
20994,884113_2071042,884113,1.309388e+08,0.315789,69600151.65,3.008146e+05,2071042,6.960015e+07,30,1.0,46,330544.323616,318936.375925,28226.751978


In [ ]:
resp.status_code, resp.headers, df_res.shape
